In [1]:
import scipy.io as io
import numpy as np

In [2]:
dataset = io.loadmat('PaviaU.mat')['paviaU']
gt_labels = io.loadmat('PaviaU_gt.mat')['paviaU_gt']
print(dataset[0][0])
print(gt_labels[0][0])

[ 647  499  464  371  291  319  365  322  296  305  277  219  219  222
  201  162  157  183  204  194  198  216  249  284  294  322  338  342
  336  342  362  365  348  341  324  316  293  274  251  244  263  248
  236  254  255  240  223  203  202  191  185  200  203  181  168  174
  185  187  165  151  144  140  149  169  192  227  287  376  493  633
  763  913 1149 1442 1759 2102 2425 2689 2895 3058 3196 3252 3195 3297
 3542 3550 3537 3545 3514 3477 3468 3433 3408 3420 3416 3335 3256 3226
 3205 3210 3221 3238 3250]
0


In [3]:
a = {x:[] for x in range(1,10)}
(x,y,z) = dataset.shape
for i in range(x):
    for j in range(y):
        l = gt_labels[i][j]
        if l!=0:
            a[l].append(np.array(dataset[i][j]))

In [4]:
for i in range(1,10):
    print(len(a[i]))

6631
18649
2099
3064
1345
5029
1330
3682
947


In [5]:
y = []
X = []
for i in range(1,10):
    for j in range(len(a[i])):
        X.append(a[i][j])
        y.append(i)
X = np.array(X)
y = np.array(y)

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from keras.utils import to_categorical
X = normalize(X,axis=1,return_norm=False)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_test = X_test.reshape(len(X_test),103,1,1)
X_train = X_train.reshape(len(X_train),103,1,1)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_train = y_train[:,1:]
y_test = y_test[:,1:]

Using TensorFlow backend.


In [7]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D , Flatten, MaxPooling2D
from keras.initializers import RandomUniform

In [10]:
from keras.initializers import RandomUniform
initializer = RandomUniform(minval=-0.05, maxval=0.05)
model = Sequential()
model.add(Conv2D(20, kernel_size=(11,1), activation='tanh', input_shape=(103,1,1),kernel_initializer=initializer,bias_initializer='zeros'))
model.add(MaxPooling2D(pool_size = (3,1)))
model.add(Flatten())
model.add(Dense(100,activation='tanh',kernel_initializer=initializer))
model.add(Dense(9,activation = 'softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 93, 1, 20)         240       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 1, 20)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 620)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               62100     
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 909       
Total params: 63,249
Trainable params: 63,249
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics = ['accuracy'])

In [12]:
model.fit(X_train,y_train,validation_data = (X_test,y_test),epochs = 20,verbose=1,batch_size=200)

Train on 28659 samples, validate on 14117 samples
Epoch 1/20
28659/28659 [==============================] - 2s 72us/step - loss: 1.2442 - acc: 0.5602 - val_loss: 0.8296 - val_acc: 0.6606
Epoch 2/20
28659/28659 [==============================] - 2s 70us/step - loss: 0.7634 - acc: 0.6784 - val_loss: 0.7332 - val_acc: 0.7172
Epoch 3/20
28659/28659 [==============================] - 2s 70us/step - loss: 0.6984 - acc: 0.7082 - val_loss: 0.6904 - val_acc: 0.7320
Epoch 4/20
28659/28659 [==============================] - 2s 75us/step - loss: 0.6645 - acc: 0.7280 - val_loss: 0.6615 - val_acc: 0.7342
Epoch 5/20
28659/28659 [==============================] - 2s 71us/step - loss: 0.6421 - acc: 0.7381 - val_loss: 0.6404 - val_acc: 0.7301
Epoch 6/20
28659/28659 [==============================] - 2s 78us/step - loss: 0.6204 - acc: 0.7505 - val_loss: 0.6193 - val_acc: 0.7548
Epoch 7/20
28659/28659 [==============================] - 2s 69us/step - loss: 0.6000 - acc: 0.7633 - val_loss: 0.6041 - val_acc